In [ ]:
!pip install transformers
!pip install pandas
!pip install torch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from transformers import pipeline
import torch

# GPU kullanımı için cihazı ayarlama
cihaz = 0 if torch.cuda.is_available() else -1

# Entity'leri tanımlama
def varliklari_tanimla(metin, nlp_modeli):
    varliklar = nlp_modeli(metin)
    birlesmis_varliklar = []
    mevcut_varlik = ""
    mevcut_etiket = ""

    for varlik in varliklar:
        kelime = varlik['word'].replace('##', '')  # ## olan sembolleri kaldır
        if varlik['entity'].startswith('B-'):
            if mevcut_varlik:
                birlesmis_varliklar.append((mevcut_varlik.strip(), mevcut_etiket.split('-')[1]))
            mevcut_varlik = kelime
            mevcut_etiket = varlik['entity']
        elif varlik['entity'].startswith('I-') and mevcut_varlik:
            mevcut_varlik += " " + kelime
        else:
            if mevcut_varlik:
                birlesmis_varliklar.append((mevcut_varlik.strip(), mevcut_etiket.split('-')[1]))
                mevcut_varlik = ""
                mevcut_etiket = ""

    if mevcut_varlik:
        birlesmis_varliklar.append((mevcut_varlik.strip(), mevcut_etiket.split('-')[1]))

    return birlesmis_varliklar

# Duygu analizini gerçekleştirme
def duygu_analizi_yap(metin, nlp_modeli):
    if not metin:
        return "neutral"
    duygu = nlp_modeli(metin)
    etiket_haritasi = {
        "LABEL_2": "positive",
        "LABEL_0": "negative",
        "LABEL_1": "neutral"
    }
    return etiket_haritasi.get(duygu[0]['label'], "neutral")

# Yorumları okuma ve işleme fonksiyonu
def yorumlari_isle(girdi_dosya_adi, varlik_model_dizini, duygu_model_dizini, cikti_dosya_adi):
    # xlsx dosyasından veri okuma
    df = pd.read_excel(girdi_dosya_adi)

    # NaN değerlerini içeren satırları silme
    df.dropna(subset=["Yorum"], inplace=True)

    nlp_varlik = pipeline("ner", model=varlik_model_dizini, tokenizer=varlik_model_dizini, device=cihaz)
    nlp_duygu = pipeline("sentiment-analysis", model=duygu_model_dizini, tokenizer=duygu_model_dizini, device=cihaz)

    yeni_satirlar = []

    for i, satir in df.iterrows():
        yorum_metni = satir["Yorum"]
        varliklar = varliklari_tanimla(yorum_metni, nlp_varlik)

        if varliklar:
            for varlik, kategori in varliklar:
                duygu = duygu_analizi_yap(varlik, nlp_duygu)
                yeni_satir = satir.copy()
                yeni_satir["Entities"] = varlik
                yeni_satir["Categories"] = kategori
                yeni_satir["Sentiments"] = duygu
                yeni_satirlar.append(yeni_satir)
        else:
            yeni_satir = satir.copy()
            yeni_satir["Entities"] = "yok"
            yeni_satir["Categories"] = "yok"
            yeni_satir["Sentiments"] = "neutral"
            yeni_satirlar.append(yeni_satir)

    yeni_df = pd.DataFrame(yeni_satirlar)

    # 'Categories' sütunu "yok" olan satırları silme
    yeni_df = yeni_df[yeni_df["Categories"] != "yok"]

    yeni_df.to_excel(cikti_dosya_adi, index=False)

# Dosya ve model dizinleri
girdi_xlsx = "/content/redmiBudsEssential.xlsx"
cikti_xlsx = 'redmiBudsEssential_duzen.xlsx'
varlik_model_dizini = "/content/drive/MyDrive/ner"
duygu_model_dizini = "/content/drive/MyDrive/duygu"

# Yorumları işleme ve yeni dosyaya yazma
yorumlari_isle(girdi_xlsx, varlik_model_dizini, duygu_model_dizini, cikti_xlsx)
